In [ ]:
import os
from os.path import join
import numpy as np
import cv2
import matplotlib.pyplot as plt
import functions as f

In [ ]:
# Creating datasets of jigsaw sets
jigsaw_1 = os.listdir('Dataset/Jigsaw_1')
jigsaw_2 = os.listdir('Dataset/Jigsaw_2')
jigsaw_3 = os.listdir('Dataset/Jigsaw_3')
jigsaw_4 = os.listdir('Dataset/Jigsaw_4')

In [ ]:
for piece in jigsaw_2:
    piece_path = os.path.join('Dataset/Jigsaw_2', piece)
    I = cv2.imread(piece_path)
    if I is not None:
        U = f.transform_colorspace(I)
        mask = f.transform_blur_and_thresh(U)
        result = f.segment_contours_cropping(mask, I)
        
        if isinstance(result, tuple) and len(result) == 2:
            if isinstance(result[0], np.ndarray) and isinstance(result[1], np.ndarray):
                pieces = [result]
            else:
                pieces = result
        else:
            pieces = result
        
        print(f"Processing: {piece} - Found {len(pieces)} puzzle pieces")
        
        for idx, (cropped_img, cropped_mask) in enumerate(pieces):
            boundary = f.piece_boundary(cropped_mask)
            if boundary is not None:
                corners = f.detect_corners(boundary)
                boundary_piece = cropped_img.copy()

                cv2.drawContours(boundary_piece, [boundary], -1, (0,255,0), 2)

                if corners is not None:
                    cv2.drawContours(boundary_piece, [corners], -1, (255,0,0), 3)
             
                    for corner in corners:
                        x, y = corner[0]
                        cv2.circle(boundary_piece, (x, y), 5, (0,0,255), -1)

                plt.imshow(cv2.cvtColor(boundary_piece, cv2.COLOR_BGR2RGB))
                plt.axis('off')
                plt.title(f"{piece} - Piece {idx}")
                plt.show()

            # print(f"  Piece {idx}: shape={cropped_img.shape}")
            # #final = f.transform_convert_clean(cropped_img, cropped_mask)
            # plt.imshow(cv2.cvtColor(cropped_mask, cv2.COLOR_BGR2RGB))
            # plt.axis('off')
            # plt.title(f"{piece} - Piece {idx}")
            # plt.show()
    else:
        print(f"Could not read image: {piece_path}")